In [1]:
import os
import itertools
import functools
import tomli
import pathlib
import glob
import re

In [2]:
import numpy as np
import pandas as pd

In [3]:
import statsmodels.api as sm
import scipy as sc

In [4]:
import matplotlib.pyplot as plt

In [5]:
import mne
import mne_nirs

In [6]:
import mbll

In [7]:
import utils
import constants
from nirs import NIRS

In [8]:
mne.set_log_level('WARNING')

In [9]:
nirs = NIRS().read(1, 1, 4)

/home/pulkitgoyal56/fNIRS-attentional-load/nirs.py:398: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  self.raw.set_montage(mne.channels.read_custom_montage(self.montage_file_path, coord_frame='mri'))


In [10]:
nirs.process(
    # Convert raw (CW amplitude) to optical density (OD) signals
        NIRS.wrap(mne.preprocessing.nirs.optical_density),
    # Motion artifact removal -- Temporal Derivative Distribution Repair (TDDR)
        NIRS.wrap(mne.preprocessing.nirs.tddr),
    # Short-channel regression
        NIRS.wrap(mne_nirs.signal_enhancement.short_channel_regression, max_dist=constants.DEVICE.SS_MAX_DIST),
    # Optical Densities -> HbO and HbR concentrations -- Modified Beer Lambert Law (MBLL)
        # NIRS.wrap(mne.preprocessing.nirs.beer_lambert_law, ppf=0.1),
        NIRS.wrap(mbll.modified_beer_lambert_law),
    # Pick long channels
        NIRS.wrap(mne_nirs.channels.get_long_channels, min_dist=constants.DEVICE.SS_MAX_DIST, max_dist=constants.DEVICE.LS_MAX_DIST),
    # Filter frequencies outside haemodynamic response range
        NIRS.wrap(mne.filter.FilterMixin.filter, 0.02, 0.4, h_trans_bandwidth=0.2, l_trans_bandwidth=0.01),
    # Negative correlation enhancement
        NIRS.wrap(mne_nirs.signal_enhancement.enhance_negative_correlation),
    # Get epochs
        NIRS.get_epochs,
    # Block average
        NIRS.block_average
)